In [463]:
import numpy as np 
import pandas as pd

In [464]:
from operator import index

#Previous ds with only the info linked to the last CT scan 
CT_cond = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\ds_annex_study.csv")

#Final ds, only containing ct scan dates and IDS at the beginning 
ds = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\ds_CPA_study.csv")

#file with all the biomarkers entries for all the patients
all_data = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\CPA imaging data fields_anonuptodate.csv")

#Version of all_data without duplicates
all_data_unique = all_data.drop_duplicates(subset='ID')

#All CT-scan info
all_CT = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\cpa_annotation_2020.csv")

#Afonso score
mort_scores = pd.read_csv(r"C:\Users\USER\OneDrive\Summer_project\Azure\annex study\new_ds\scores_afonso_raw_ELSA.csv")

In [465]:
#Indicate the CT-scan number for each patient 
ds.insert(1, 'CT-SCAN', ds.ID.str[-1])
ds['fullID'] = ds.ID
#Removing the CT-scan number information from the ID to facilitate joints with the other ds
ds.ID = ds.ID.str[:-2]

#Edge case
ds.ID[ds.ID == 'RBH CPA 13/'] = ds.ID.str[:-1]


#Merge from all values - demographic 
ds = ds.merge(all_data_unique[['sex', 'ID', 'DOB', 'ETHNIC', 'death_date']], on='ID',how="left")
ds.DOB = ds.DOB.str[:-6]
ds = ds.merge(CT_cond[['ID','COPD','ILD','NTM']],on='ID',how="left")
print(ds)


             ID CT-SCAN        date        fullID sex         DOB ETHNIC  \
0     RBH CPA 1       1  16/09/2016   RBH CPA 1/1   F  07/12/1951    NaN   
1     RBH CPA 1       2  16/09/2016   RBH CPA 1/2   F  07/12/1951    NaN   
2     RBH CPA 3       2  07/10/2015   RBH CPA 3/2   F  22/01/1942      A   
3     RBH CPA 3       3  25/11/2016   RBH CPA 3/3   F  22/01/1942      A   
4     RBH CPA 3       5  15/03/2017   RBH CPA 3/5   F  22/01/1942      A   
..          ...     ...         ...           ...  ..         ...    ...   
228  RBH CPA 94       1  05/02/2015  RBH CPA 94/1   F  03/08/1950      B   
229  RBH CPA 96       1  23/11/2016  RBH CPA 96/1   F  11/12/1979      J   
230  RBH CPA 97       1  13/03/2013  RBH CPA 97/1   F  02/01/1952      Z   
231  RBH CPA 98       1  19/07/2017  RBH CPA 98/1   M  05/03/1942      A   
232  RBH CPA 99       1  20/11/2017  RBH CPA 99/1   M  20/09/1954      A   

    death_date COPD ILD  NTM  
0          NaN   No  No   No  
1          NaN   No  No  

In [466]:
#The age is taken to be the age of the patient at the time where the CT-scan was recorded
ds.age = ((pd.to_datetime(ds.date, dayfirst=True)) - (pd.to_datetime(ds.DOB,  dayfirst=True))) / np.timedelta64(1, 'Y')
ds = ds.drop(columns = 'DOB')

C:\Users\USER\AppData\Local\Temp\ipykernel_11800\4247691355.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ds.age = ((pd.to_datetime(ds.date, dayfirst=True)) - (pd.to_datetime(ds.DOB,  dayfirst=True))) / np.timedelta64(1, 'Y')


In [467]:
#Defining patients statuts after 5 years - 0: alive, 1: dead, nan: unknown, a value of 0 is still attributed 
#to patients alive at the last db update but with ct +  5y not reached as they will be right censored 


ds.fy = pd.to_datetime(pd.to_datetime(ds.date, dayfirst=True) + pd.offsets.DateOffset(years=5))
ds['death5'] = 0
ds['death5'][ds.fy  > ds.death_date] = 1
# ds['death5'][ds.fy  > pd.to_datetime('01-05-2021')] =  np.nan



C:\Users\USER\AppData\Local\Temp\ipykernel_11800\2344255170.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ds.fy = pd.to_datetime(pd.to_datetime(ds.date, dayfirst=True) + pd.offsets.DateOffset(years=5))
C:\Users\USER\AppData\Local\Temp\ipykernel_11800\2344255170.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['death5'][ds.fy  > ds.death_date] = 1


In [468]:
#Collecting variables closest to the CT scan for each patient 
all_data.sampleCollectDateTime = pd.to_datetime(all_data.sampleCollectDateTime)

#add a closest variable that stores the closest available biomarker entry
ds['closest'] = 0

#Find the closest date between a goal date (date of CT scan in our case) and a set of dates (all biomarker entries for a patient in our case)
def closer_date(set_of_dates, goal):
    return min(set_of_dates, key=lambda x: abs(x - goal))

#find the best observation date for each 'patient'
#(1) find the date of observation for each patient 
date_set = []
for index, row in ds.iterrows():
    date_set = np.array(all_data[all_data.ID == row.ID].sampleCollectDateTime.values)
    
    date = pd.to_datetime(row.date , dayfirst=True)
    if np.isnan(date_set).all():
        closest_date = np.nan
    else:
        closest_date = closer_date(date_set, date)
      
    ds.loc[index,'closest'] = closest_date

#Question: parfois y'a des dates ou y'a pas de biomarkers?? Que faire?? -- A vérifier manuellement 

C:\Users\USER\AppData\Local\Temp\ipykernel_11800\788907502.py:2: UserWarning: Parsing '13/09/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  all_data.sampleCollectDateTime = pd.to_datetime(all_data.sampleCollectDateTime)
C:\Users\USER\AppData\Local\Temp\ipykernel_11800\788907502.py:2: UserWarning: Parsing '18/05/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  all_data.sampleCollectDateTime = pd.to_datetime(all_data.sampleCollectDateTime)
C:\Users\USER\AppData\Local\Temp\ipykernel_11800\788907502.py:2: UserWarning: Parsing '24/04/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  all_data.sampleCollectDateTime = pd.to_datetime(all_data.sampleCollectDateTime)
C:\Users\USER\AppData\Local\Temp\ipykernel_11800\788907502.py:2: UserWarning: Parsing '16/05/2017' in DD/MM/YYYY format. Provide format or specify infer_datetim

In [469]:
#add entries 
ds['crp'] = 0
ds['albumin'] = 0
for index, row in ds.iterrows():
    date = row['closest']
    crp_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].crp.values
    albumin_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].albumin.values

    if len(crp_val) > 1:
        crp_val = crp_val[0]
    elif len(crp_val) == 0:
        crp_val = np.nan
    if len(albumin_val) > 1:
        albumin_val = albumin_val[0]
    elif len(albumin_val) == 0:
        albumin_val = np.nan
    ds.loc[index, 'crp'] = crp_val
    ds.loc[index, 'albumin'] = albumin_val


C:\Users\USER\AppData\Local\Temp\ipykernel_11800\4024585698.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  crp_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].crp.values
C:\Users\USER\AppData\Local\Temp\ipykernel_11800\4024585698.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  albumin_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].albumin.values


In [470]:
#Same for BMI 
#Collecting variables closest to the CT scan for each patient 
all_data.weight_measurement_dttm = pd.to_datetime(all_data.weight_measurement_dttm)

#add a closest variable that stores the closest available biomarker entry
ds['closest'] = 0


#find the best observation date for each 'patient'
#(1) find the date of observation for each patient 
date_set = []
for index, row in ds.iterrows():
    date_set = np.array(all_data[all_data.ID == row.ID].weight_measurement_dttm.values)
    date = pd.to_datetime(row.date , dayfirst=True)

    if pd.isnull(date_set).all():
        closest_date = np.nan
    elif pd.isnull(date_set).any():
        date_set = date_set[~np.isnat(date_set)]
        closest_date = closer_date(date_set, date)
    else:
        closest_date = closer_date(date_set, date)
    ds.loc[index,'closest'] = closest_date


c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/09/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/05/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/03/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\panda

In [471]:
#BMI
ds['BMI'] = 0
for index, row in ds.iterrows():
    date = row['closest']
    BMI_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].BMI.values

    if len(BMI_val) > 1:
        BMI_val = BMI_val[0]
    elif len(BMI_val) == 0:
        BMI_val = np.nan
    
    ds.loc[index, 'BMI'] = BMI_val


C:\Users\USER\AppData\Local\Temp\ipykernel_11800\493107805.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  BMI_val = all_data[all_data.ID == row.ID][all_data.sampleCollectDateTime == date].BMI.values


In [472]:
all_CT['fullID'] = all_CT.ID
# print(all_CT.columns)
ds = ds.merge(all_CT[['fullID', 'preexisting_cond', 'cavities', 'fungal_balls']], on='fullID',how="left")

In [473]:
#1-hot encoding the preexisting conds
ds[['FL', 'FB', 'SB', 'BB', 'BR',
       'BL', 'ER', 'EB', 'GR', 'GL', 'GB', 'CB', 'CR', 'CL']]= pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

list_cond = ['FL', 'FB', 'SB', 'BB', 'BR',
       'BL', 'ER', 'EB', 'GR', 'GL', 'GB', 'CB', 'CR', 'CL']
for index, row in ds.iterrows():
       for i in list_cond:
              ds.loc[index, i] = 0
              if i in ds.loc[index,'preexisting_cond']:
                     ds.loc[index, i] = 1

In [474]:
#Presence of balls or cavities (lateral or bilateral)
list = ['cavities', 'fungal_balls']
for index, row in ds.iterrows():
       for i in list:
              if  ds.loc[index, i] == 0:
                     ds.loc[index, i] = 0 
              elif 'R' in ds.loc[index, i] and 'L' in ds.loc[index, i]:
                     ds.loc[index, i] = 2  
              elif 'R' in ds.loc[index, i] or 'L' in ds.loc[index, i]:
                     ds.loc[index, i] = 1  
              else:
                     ds.loc[index, i] = 0 
ds["summary"] = ds[["cavities", "fungal_balls"]].max(axis=1)

In [475]:
#define survival time in days
ds['surv'] = 0
for index, row in ds.iterrows():
    if pd.to_datetime(row.death_date) < pd.to_datetime('01-05-2021'):
        surv = pd.to_datetime(row.death_date) - pd.to_datetime(row.date, dayfirst=True)
        ds.loc[index, 'surv'] = surv
    else:
        surv =ds.loc[index, 'surv'] = pd.to_datetime('01-05-2021') - pd.to_datetime(row.date, dayfirst=True)
        ds.loc[index, 'surv'] = pd.to_datetime('01-05-2021') - pd.to_datetime(row.date, dayfirst=True)


In [476]:
#incorporation of Afonso score
# print(ds)
ds['score'] = 0
for index, row in ds.iterrows():
    nCT = int(row['CT-SCAN'])
    scores = mort_scores[mort_scores.ID == row.ID].iloc[:,1:1+nCT].values 
    ds.loc[index, 'score'] = np.mean(scores)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [477]:
#save ds
ds.to_csv('preprocessed_cpa_csv')